In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://cloud.

In [2]:
# Download a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-27 18:22:48--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.22MB/s    in 0.2s    

2022-08-27 18:22:48 (6.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Add drivers to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

**Tweets**

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://uaresources.s3.amazonaws.com/aug_twitter_sentiments.csv"

spark.sparkContext.addFile(url)
tweets_df = spark.read.csv(SparkFiles.get("aug_twitter_sentiments.csv"), sep=",", header=True, inferSchema=True)


# Show DataFrame
tweets_df.show()

+-------------------+--------------------+--------------------+-------------------+--------------------+
|     tweetcreatedts|        cleaned_text|            negative|            neutral|            positive|
+-------------------+--------------------+--------------------+-------------------+--------------------+
|2022-08-01 00:00:00|the conflict is b...|  0.6027116775512695|0.38163381814956665|  0.0156544279307127|
|2022-08-01 00:00:00|remember when sma...| 0.21225015819072723| 0.7508949041366577|0.036854907870292664|
|2022-08-01 00:00:01|hi  you have to u...|  0.4901295006275177|   0.46605184674263|0.043818794190883636|
|2022-08-01 00:00:01|the world is in d...|  0.8760228157043457|0.11690670996904373|0.007070485968142748|
|2022-08-01 00:00:01|will the imposed ...| 0.04450933262705803| 0.9108507633209229| 0.04463988170027733|
|2022-08-01 00:00:01|owner of ukraines...|  0.8333820104598999| 0.1629866510629654|  0.0036314248573035|
|2022-08-01 00:00:01|animal shelter do...|0.01331210788

In [5]:
# Import pandas
import pandas as pd

#Import datetime
import datetime as dt

In [6]:
# Change column name tweetcreatedts to Date
tweets_df = tweets_df.withColumnRenamed("tweetcreatedts", "Date")

# Change date format 
# tweets_df = tweets_df.dt.strftime('%Y-%m-%d')

# Show DataFrame
tweets_df.show()

+-------------------+--------------------+--------------------+-------------------+--------------------+
|               Date|        cleaned_text|            negative|            neutral|            positive|
+-------------------+--------------------+--------------------+-------------------+--------------------+
|2022-08-01 00:00:00|the conflict is b...|  0.6027116775512695|0.38163381814956665|  0.0156544279307127|
|2022-08-01 00:00:00|remember when sma...| 0.21225015819072723| 0.7508949041366577|0.036854907870292664|
|2022-08-01 00:00:01|hi  you have to u...|  0.4901295006275177|   0.46605184674263|0.043818794190883636|
|2022-08-01 00:00:01|the world is in d...|  0.8760228157043457|0.11690670996904373|0.007070485968142748|
|2022-08-01 00:00:01|will the imposed ...| 0.04450933262705803| 0.9108507633209229| 0.04463988170027733|
|2022-08-01 00:00:01|owner of ukraines...|  0.8333820104598999| 0.1629866510629654|  0.0036314248573035|
|2022-08-01 00:00:01|animal shelter do...|0.01331210788

In [7]:
# Save the tweets_df as a csv file.
tweets_df.toPandas().to_csv('cleanedtweets.csv', encoding='utf-8',header=True)

# tweets_df.toPandas().to_csv('cleanedtweets.csv')

KeyboardInterrupt: ignored

In [ ]:
# Veronica this is the command to download the csv from colab -Jesse
from google.colab import files
files.download('cleanedtweets.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Events**

In [10]:
# Read in data from S3 Buckets
url ="https://uaresources.s3.amazonaws.com/EventsUA.csv"
spark.sparkContext.addFile(url)
event_df = spark.read.csv(SparkFiles.get("EventsUA.csv"), sep=",", header=True, inferSchema=True)

# Change column name tweetcreatedts to Date
event_df = event_df.withColumnRenamed("Date", "Date")

# Show DataFrame
event_df.show()

+----+-------+--------------------+------+
| _c0|   Date|               Event|Weight|
+----+-------+--------------------+------+
|null|2/24/22|Start Russian Inv...|   1.0|
|null|2/25/22|Battles for Kyiv,...|  0.75|
|null| 3/1/22|Bombing of Babiy ...|  0.75|
|null| 3/3/22|Russian attack fo...|  0.75|
|null| 3/6/22|Bombing of the th...|  0.75|
|null| 3/9/22|Bombing of the ma...|  0.75|
|null|3/13/22|Documentary maker...|   0.3|
|null|3/24/22|Surviving prisone...|   0.3|
|null| 4/3/22|Evidence of massa...|  0.75|
|null| 4/8/22|The shelling of t...|  0.75|
|null| 4/9/22|British Prime Min...|   0.3|
|null|4/14/22|Sinking of the Ru...|   0.5|
|null|4/28/22|The U.S. House of...|   0.5|
|null|5/16/22|Exit of the wound...|   0.3|
|null|6/27/22|Rocket attack on ...|   0.5|
|null|7/28/22|Torture and murde...|   0.3|
|null| 8/1/22|In Kharkov, shell...|  null|
|null| 8/2/22|Russian Armed For...|  null|
|null|8/12/22|�Battles in easte...|   0.1|
|null|8/11/22|Ukraine and Russi...|   0.1|
+----+-----

In [9]:
# Join the two DataFrame
joined_df=tweets_df.join(event_df, on='Date', how='left')
joined_df.show()

+-------------------+--------------------+--------------------+-------------------+--------------------+----+-----+------+
|               Date|        cleaned_text|            negative|            neutral|            positive| _c0|Event|Weight|
+-------------------+--------------------+--------------------+-------------------+--------------------+----+-----+------+
|2022-08-01 00:00:00|the conflict is b...|  0.6027116775512695|0.38163381814956665|  0.0156544279307127|null| null|  null|
|2022-08-01 00:00:00|remember when sma...| 0.21225015819072723| 0.7508949041366577|0.036854907870292664|null| null|  null|
|2022-08-01 00:00:01|hi  you have to u...|  0.4901295006275177|   0.46605184674263|0.043818794190883636|null| null|  null|
|2022-08-01 00:00:01|the world is in d...|  0.8760228157043457|0.11690670996904373|0.007070485968142748|null| null|  null|
|2022-08-01 00:00:01|will the imposed ...| 0.04450933262705803| 0.9108507633209229| 0.04463988170027733|null| null|  null|
|2022-08-01 00:0

In [ ]:
# Drop null values
dropna_df=joined_df.dropna()
dropna_df.show()

+----+------------+--------+-------+--------+---+-----+------+
|Date|cleaned_text|negative|neutral|positive|_c0|Event|Weight|
+----+------------+--------+-------+--------+---+-----+------+
+----+------------+--------+-------+--------+---+-----+------+



In [ ]:
# Looking at joined_df again since dropna_df came up with nothing 
joined_df.show()

+-------------------+--------------------+--------------------+-------------------+--------------------+----+-----+------+
|               Date|        cleaned_text|            negative|            neutral|            positive| _c0|Event|Weight|
+-------------------+--------------------+--------------------+-------------------+--------------------+----+-----+------+
|2022-08-01 00:00:00|the conflict is b...|  0.6027116775512695|0.38163381814956665|  0.0156544279307127|null| null|  null|
|2022-08-01 00:00:00|remember when sma...| 0.21225015819072723| 0.7508949041366577|0.036854907870292664|null| null|  null|
|2022-08-01 00:00:01|hi  you have to u...|  0.4901295006275177|   0.46605184674263|0.043818794190883636|null| null|  null|
|2022-08-01 00:00:01|the world is in d...|  0.8760228157043457|0.11690670996904373|0.007070485968142748|null| null|  null|
|2022-08-01 00:00:01|will the imposed ...| 0.04450933262705803| 0.9108507633209229| 0.04463988170027733|null| null|  null|
|2022-08-01 00:0

**Get our transformed raw data into our database - Don't need this as we already have a pgAdmin 4 Database**

In [21]:
# Store environmental variable
from getpass import getpass
password = getpass('password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://tweets.cnzbbvrrhst7.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"uatweets",
          "password": password,
          "driver":"org.postgresql.Driver"}

password··········


In [22]:
# Write DataFrame to tweets_df table in RDS
tweets_df.write.jdbc(url=jdbc_url, table='test', mode=mode, properties=config)